<div class="usecase-title">Live Music Event Planning</div>

<div class="usecase-authors"><b>Authored by: </b> Mollie Fernandez </div>

<div class="usecase-duration"><b>Duration:</b> {90} mins</div>

<div class="usecase-level-skill">
    <div class="usecase-level"><b>Level: </b>{Beginner}</div>
    <div class="usecase-skill"><b>Skills: </b>{Python, Data Manipulation, Folium}</div>
</div>

# <div class="usecase-section-header">Scenario</div>

**From an Music manager's perspective:** Tasked with representing multiple musicians, obtaining insights into available venues for booking shows in Melbourne is paramount. Understanding the proximity of these venues to publically accessible advertising spaces is crucial for effective promotion. By leveraging comprehensive data analysis and visualisation techniques, we can streamline the venue selection process and identify strategic locations for advertising campaigns, optimising exposure for live music events.


**From a council perspective:** Facilitating the planning and promotion of live music events within Melbourne city presents an opportunity to support both venues and musicians alike. By providing access to integrated datasets on venue availability, public spaces, and cultural activities, the council can empower stakeholders with valuable insights for event planning and promotion strategies. Fostering collaboration between local authorities, venue operators, and musicians can facilitate the development of initiatives aimed at promoting Melbourne's vibrant music scene and drive economic growth within the community.

# Objectives for the Exploratory Data Analysis

By leveraging data aggregation, we can merge Melbourne's Open datasets including Bookable Banner poles and Live Music Venues, offering insights into potential event locations and advertisement placements. Through the integration of open government datasets, we aim to observe, analyse, and report on geographical patterns between these datasets.


**Key questions guiding our analysis include:**
Where are suitable venues for hosting events? How many nearby banner poles are available for event advertisement? These inquiries drive our exploration, ensuring informed decision-making in event management.


Our goals for exploratory data analysis are to identify live music venues and advertisement opportunities. By adopting a professional yet accessible approach, we seek to provide actionable insights that streamline event planning and enhance promotional strategies for live music events in Melbourne's dynamic cultural scene.

# Importing Dependancies
All of these libraries will help us to visualise our data and display it in a way which will make it easier to interpret.

In [ ]:
import sys
import warnings
warnings.simplefilter("ignore")
import requests
import pandas as pd
import numpy as np
import geopandas
import json
import plotly.express as px
import folium
import os
from google.colab import drive
from io import StringIO
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Importing datasets with API
Using an API offers benefits like real-time data access, structured data retrieval, enhanced security, scalability, versioning support, cross-platform compatibility, and features like rate limiting. These advantages make APIs preferable over direct file access for accessing data in various applications.

API keys can be generated throught the City of Melbourne Open Data Platform by creating an account and generating a key.
https://data.melbourne.vic.gov.au/account/api-keys/

For security reasons we use google drive to call our API Key as we don't want everyone to know what our key is.

In [ ]:
# Mount our drive and give access to our Google Drive files
#drive.mount('/content/drive')
#with open('/content/drive/My Drive/API_KEY.txt', 'r') as file:
#    api_key = file.read().strip()

#api_key = os.getenv(api_key)

### Calling the datasets

We use the City of Melbourne Open Data URL as the base where we collect all of our data.
We fist collect the Bookable Banner Poles dataset.
We then collect the Live Music Venues dataset.

In [ ]:
# Here we define the City of Melbourne Open Data base URL where we will be getting our datasets
base_url = 'https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'
#apikey = api_key

In [ ]:
#https://data.melbourne.vic.gov.au/explore/dataset/bookable-banner-poles/information/
data2 = 'bookable-banner-poles'

dataset_id2 = data2
format = 'csv'

url = f'{base_url}{dataset_id2}/exports/{format}'
params = {
    'select': '*',
    'limit': -1,  # all records
    'lang': 'en',
    'timezone': 'UTC',
    #'api_key': apikey
}

# GET request
response = requests.get(url, params=params)

if response.status_code == 200:
    # StringIO to read the CSV data
    url_content = response.content.decode('utf-8')
    banner_data = pd.read_csv(StringIO(url_content), delimiter=';')
    print(banner_data.sample(10, random_state=999)) # Test
else:
    print(f'Request failed with status code {response.status_code}')


     asset_number                                  asset_description  \
135       1110444  Standard Banner Pole - Bourke St: William St -...   
490       1110665  Docklands Type 1 Banner Pole - Central Pier: O...   
503       1110565  Standard Banner Pole - Clarendon St: Flinders ...   
55        1110328  Standard Banner Pole - Spencer St:  Lonsdale S...   
260       1110259  Standard Banner Pole - Elizabeth St: Flinders ...   
687       1109902                    Banner Pole - Super Banner Pole   
139       1109975  Super Banner Pole - Spencer St: Corner Flinder...   
556       1110365  Standard Banner Pole - Lansdowne St: Wellingto...   
311       1110563  Standard Banner Pole - Clarendon St: Flinders ...   
567       1110193  Standard Banner Pole - Collins St: Elizabeth S...   

     banner_pole_type  book_group              model_descr_lupvalue  \
135               6.0        12.0     Banner Pole - Standard Banner   
490               4.0        30.0  Banner Pole - Type 1 - Docklan

In [ ]:
# https://data.melbourne.vic.gov.au/explore/dataset/live-music-venues/information/
data1 = 'live-music-venues'

dataset_id1 = data1
format = 'csv'

url = f'{base_url}{dataset_id1}/exports/{format}'
params = {
    'select': '*',
    'limit': -1,  # all records
    'lang': 'en',
    'timezone': 'UTC',
    #'api_key': apikey
}

# GET request
response = requests.get(url, params=params)

if response.status_code == 200:
    # StringIO to read the CSV data
    url_content = response.content.decode('utf-8')
    venue_data = pd.read_csv(StringIO(url_content), delimiter=';')
    print(venue_data.sample(10, random_state=999)) # Test
else:
    print(f'Request failed with status code {response.status_code}')


     property_number                                     venue_name  \
110           110477                               Level 3 at Crown   
73            681564                                   Speakeasy HQ   
40            597955                                      AAMI Park   
42            621998  Australian Centre for the Moving Image (ACMI)   
26            105449                             Duke Of Kent Hotel   
144           657233                               The Toff in Town   
207           622423                    Groove Live Bar and Terrace   
183           103402                                Town Hall Hotel   
141           562238                                      Manchuria   
118           652991                       Sheraton Hotel Melbourne   

                                         venue_address         space_type  \
110                 8 Whiteman Street, Southbank, 3006          Nightclub   
73   Level 1, 520-522 Flinders Street, Melbourne, 3000          

# Bookable Banner Dataset

We will start with the Bookable Banner Poles dataset and learn how to prepare the data for how we want to display it.

### Viewing Dataset

First we want to look at the dataset to see what we are working with.

In [ ]:
# View banner dataset
banner_data.head()

,asset_number,asset_description,banner_pole_type,book_group,model_descr_lupvalue,model_no_lupvalue,lat,lon,location
0,1110368,Standard Banner Pole - Lansdowne St: Wellingto...,6.0,62.0,Banner Pole - Standard Banner,310101-004,-37.814268,144.977428,"-37.81426836404757, 144.97742756833915"
1,1110057,Banner Pole - Super Banner Pole,2.0,79.0,Banner Pole - Super Banner Pole,310101-001,-37.809428,144.966548,"-37.80942811378078, 144.96654780374206"
2,1110093,Banner Pole - Super Banner Pole,2.0,59.0,Banner Pole - Super Banner Pole,310101-001,-37.800985,144.957241,"-37.80098530840636, 144.9572409533404"
3,1110096,Banner Pole - Super Banner Pole,2.0,59.0,Banner Pole - Super Banner Pole,310101-001,-37.800974,144.957269,"-37.80097426523944, 144.95726931044308"
4,1110108,Banner Pole - Super Banner Pole,2.0,59.0,Banner Pole - Super Banner Pole,310101-001,-37.801019,144.957138,"-37.801019397609785, 144.95713847866782"


### Data Processing

The next step is to remove the columns that we won't be using duirng this analysis. For this data set that means removing the 'location', 'model_no_lupalue', 'asset_description' and 'model_descr_lupvalue' columns as they are not relevant.

In [ ]:
data = banner_data
banner = pd.DataFrame(data)

# Removing columns
columns_to_remove = ['location', 'model_no_lupvalue', 'asset_description', 'model_descr_lupvalue']
banner = banner.drop(columns=columns_to_remove)

# Display the refined dataset
banner.head()

,asset_number,banner_pole_type,book_group,lat,lon
0,1110368,6.0,62.0,-37.814268,144.977428
1,1110057,2.0,79.0,-37.809428,144.966548
2,1110093,2.0,59.0,-37.800985,144.957241
3,1110096,2.0,59.0,-37.800974,144.957269
4,1110108,2.0,59.0,-37.801019,144.957138


### Convert Dataset to a GeoDataFrame
In order to use these libraries we need to ensure that our data is in the correct format. We are going to use our data to create a GeoDataFrame. What this does is combines the longitude and latitude to create a point which can be plotted on a map.

In [ ]:
# Create a GeoDataFrame from the DataFrame
geometry = [Point(lon, lat) for lon, lat in zip(banner['lon'], banner['lat'])]
geo_banner = gpd.GeoDataFrame(banner, geometry=geometry, crs="EPSG:4326")

# Drop the latitude and longitude columns if you no longer need them
geo_banner.drop(columns=['lat', 'lon'], inplace=True)

# Display the GeoDataFrame
print(geo_banner)

     asset_number  banner_pole_type  book_group                     geometry
0         1110368               6.0        62.0  POINT (144.97743 -37.81427)
1         1110057               2.0        79.0  POINT (144.96655 -37.80943)
2         1110093               2.0        59.0  POINT (144.95724 -37.80099)
3         1110096               2.0        59.0  POINT (144.95727 -37.80097)
4         1110108               2.0        59.0  POINT (144.95714 -37.80102)
..            ...               ...         ...                          ...
854       1110400               6.0        71.0  POINT (144.97323 -37.80670)
855       1110278               6.0         6.0  POINT (144.96187 -37.81453)
856       1350854               2.0       109.0  POINT (144.94242 -37.81279)
857       1109993               2.0        69.0  POINT (144.96589 -37.80674)
858       1645057               2.0        68.0  POINT (144.96680 -37.80217)

[859 rows x 4 columns]


### Visualise Banner Data Sets

Now that we have correctly prepared our data we are going to use Folium and GeoPandas packages to visualise the Bookable Banner Locations dataset.

This Python code with Folium adds GeoJSON data from geo_banner to a map. It customizes markers with star icons and configures tooltips and popups to display information like "asset_number", "banner_pole_type", and "book_group". Additionally, it applies a custom style function to the GeoJSON features. Ultimately, the code creates an interactive map with detailed feature information.

In [ ]:
# Creating the map base with Melbourne as the focal point with a 14 zoom frames the area of our dataset perfectly
map = folium.Map([-37.81368709240999, 144.95738102347036], zoom_start=14)

# Define the colours for each banner type
marker_colors = {
    '2.0': 'blue',
    '4.0': 'red',
    '5.0': 'green',
    '6.0': 'orange',
    '7.0': 'purple',
    '8.0': 'yellow',
    'nan': 'black'
}

# Defining the function to call and assign each colour according to the banner type
def style_function(feature):
    type_value = str(feature['properties']['banner_pole_type'])  # Convert to string
    color = marker_colors.get(type_value, 'black') # If no type is listed 'black' is the default
    return {'markerColor': color}

# Creation of the map
folium.GeoJson(
    geo_banner,
    name="Bookable Banner Poles",
    zoom_on_click=True, # When you click on a banner location the map will zoom and center on that location
    marker=folium.Marker(icon=folium.Icon(icon='star')), # This defines the shape of the icon and defines a 'star' within
    tooltip=folium.GeoJsonTooltip(fields=["asset_number", "banner_pole_type", "book_group"], aliases=["Asset Number", "Banner Type", "Booking Group"], labels=True, localize=True), # This describes what is displayed when the mouse hovers over a banner location
    popup=folium.GeoJsonPopup(fields=["asset_number", "banner_pole_type", "book_group"], aliases=["Asset Number", "Banner Type", "Booking Group"], labels=True, localize=True), # This describes what is displayed when a banner location is clicked on
    style_function=style_function # This calls the style function and facilitates the colours of each banner location
).add_to(map)

# Display map
map

# Live Music Venues Dataset

### View Dataset
First we want to look at the dataset to see what we are working with.

In [ ]:
#view venue dataset
venue_data.head()

,property_number,venue_name,venue_address,space_type,website,lat,lon,location,geolocation
0,609454,Alumbra,"Shed 9, Victoria Harbour, Docklands, 3008",Nightclub,http://www.alumbra.com.au/,-37.816569,144.943835,POINT (144.943834998348 -37.816568815),"-37.81656881500009, 144.94383499834845"
1,506429,Sofitel Melbourne On Collins,"Sofitel Hotel, Collins Tower, 25 Collins Stree...",Accommodation,http://www.sofitel-melbourne.com.au/,-37.813765,144.972923,POINT (144.972923487598 -37.813765267006),"-37.81376526700552, 144.9729234875978"
2,646930,Bar Nacional,"Tenancy 47, Ground, 727 Collins Street, Dockla...",Bar,http://www.barnacional.com.au/,-37.820467,144.949823,POINT (144.949822902931 -37.820467259122),"-37.82046725912159, 144.94982290293075"
3,681513,Bomba Tapas Bar and Rooftop,"Ground & Rooftop, 103 Lonsdale Street, Melbour...",Bar,https://bombabar.com.au/,-37.810381,144.969236,POINT (144.969235763373 -37.810380634625),"-37.81038063462455, 144.9692357633733"
4,565482,Melbourne Museum,"11 Nicholson Street, Carlton, 3053",Museum,https://museumsvictoria.com.au/melbournemuseum/,-37.803728,144.973731,POINT (144.973731480634 -37.803728470513),"-37.80372847051283, 144.97373148063383"


### Data Processing
The next step is to remove the columns that we won't be using duirng this analysis. For this data set that means removing the 'property_number', 'geolocation' and 'location' columns as they are not relevant.

In [ ]:
data = venue_data
venue = pd.DataFrame(data)

# Removing columns
columns_to_remove = ['property_number', 'geolocation', 'location']
venue = venue.drop(columns=columns_to_remove)

# Display refined dataset
venue.head()

,venue_name,venue_address,space_type,website,lat,lon
0,Alumbra,"Shed 9, Victoria Harbour, Docklands, 3008",Nightclub,http://www.alumbra.com.au/,-37.816569,144.943835
1,Sofitel Melbourne On Collins,"Sofitel Hotel, Collins Tower, 25 Collins Stree...",Accommodation,http://www.sofitel-melbourne.com.au/,-37.813765,144.972923
2,Bar Nacional,"Tenancy 47, Ground, 727 Collins Street, Dockla...",Bar,http://www.barnacional.com.au/,-37.820467,144.949823
3,Bomba Tapas Bar and Rooftop,"Ground & Rooftop, 103 Lonsdale Street, Melbour...",Bar,https://bombabar.com.au/,-37.810381,144.969236
4,Melbourne Museum,"11 Nicholson Street, Carlton, 3053",Museum,https://museumsvictoria.com.au/melbournemuseum/,-37.803728,144.973731


### Convert to GeoDataFrame
In order to use these libraries we need to ensure that our data is in the correct format. We are going to use our data to create a GeoDataFrame. What this does is combines the longitude and latitude to create a point which can be plotted on a map.

In [ ]:
# Create a GeoDataFrame from the DataFrame
geometry = [Point(lon, lat) for lon, lat in zip(venue['lon'], venue['lat'])]
geo_venue = gpd.GeoDataFrame(venue, geometry=geometry, crs="EPSG:4326")

# Drop the latitude and longitude columns if you no longer need them
geo_venue.drop(columns=['lat', 'lon'], inplace=True)

# Display the GeoDataFrame
print(geo_venue)

                       venue_name  \
0                         Alumbra   
1    Sofitel Melbourne On Collins   
2                    Bar Nacional   
3     Bomba Tapas Bar and Rooftop   
4                Melbourne Museum   
..                            ...   
222                    Gin Palace   
223               Rod Laver Arena   
224                West Space Inc   
225        The Metropolitan Hotel   
226             The Hotel Windsor   

                                         venue_address         space_type  \
0            Shed 9, Victoria Harbour, Docklands, 3008          Nightclub   
1    Sofitel Hotel, Collins Tower, 25 Collins Stree...      Accommodation   
2    Tenancy 47, Ground, 727 Collins Street, Dockla...                Bar   
3    Ground & Rooftop, 103 Lonsdale Street, Melbour...                Bar   
4                   11 Nicholson Street, Carlton, 3053             Museum   
..                                                 ...                ...   
222             

### Visualise Live Music Data
Now that we have correctly prepared our data we are going to use Folium and GeoPandas packages to visualise the Bookable Banner Locations dataset.

This Python code with Folium adds GeoJSON data from geo_banner to a map. It customizes markers with star icons and configures tooltips and popups to display information like "venue_address", "space_type", and "venue_name". Additionally, it applies a custom style function to the GeoJSON features. Ultimately, the code creates an interactive map with detailed feature information.

In [ ]:
# Creating the map base with Melbourne as the focal point with a 14 zoom frames the area of our dataset perfectly
map = folium.Map([-37.81368709240999, 144.95738102347036], zoom_start=14)

# Define the colours for each space type
marker_colors = {
    'Nightclub': 'blue',
    'Accommodation': 'red',
    'Bar': 'green',
    'Museum': 'orange',
    'Venue': 'purple',
    'Hotel': 'yellow',
    'Performance space': 'cyan',
    'Casino': 'magenta',
    'Amusement/Theme Venue': 'lime',
    'Multi-use': 'brown',
    'Public University/College': 'pink',
    'Gallery': 'teal',
    'Theatre': 'gray',
    'Recording': 'olive',
    'Church': 'navy',
    'Community hall': 'maroon',
    'Library': 'silver'
}

# Defining the function to call and assign each colour according to the banner type
def style_function(feature):
    type_value = str(feature['properties']['space_type'])  # Convert to string
    color = marker_colors.get(type_value, 'black') # If no type is listed 'black' is the default
    return {'markerColor': color}
# Allows the website link to be come interactive and able to be clicked on
geo_venue['website'] = '<a href="' + geo_venue.website + '">' + geo_venue.website + "</a>"

# Creation of the map
folium.GeoJson(
    geo_venue,
    name="Live Music Venues",
    zoom_on_click=True, # When you click on a banner location the map will zoom and center on that location
    marker=folium.Marker(icon=folium.Icon(icon='star')), # This defines the shape of the icon and defines a 'star' within
    tooltip=folium.GeoJsonTooltip(fields=["venue_name", "space_type", "venue_address", "website"], aliases=["Venue Name", "Type", "Address", "Website"], labels=True, localize=True, parse_html=True), # This describes what is displayed when the mouse hovers over a banner location
    popup=folium.GeoJsonPopup(fields=["venue_name", "space_type", "venue_address", "website"], aliases=["Venue Name", "Type", "Address", "Website"], labels=True, localize=True, parse_html=True), # This describes what is displayed when a banner location is clicked on
    style_function=style_function, # This calls the style function and facilitates the colours of each banner location
    ).add_to(map)

# Display map
map

# Combine Data sets and analyse

### Plot the Live Music Venues and the Bookable Banner locations together

This Python script utilises the Folium library to generate an interactive map showcasing live music venues and bookable banner locations in Melbourne City. The map is centered on Melbourne's coordinates, and its zoom level is set to perfectly frame the area of interest. Each live music venue is represented by a GeoJSON feature, with markers customized to display as stars. The color of each marker corresponds to the type of venue, as defined by the provided color dictionary. Additionally, tooltips and popups provide detailed information about each venue, including its name, type, address, and website link. The map offers interactivity, allowing users to zoom in on specific venues and access additional information by clicking on the markers. Overall, this enables the visualisation and exploration of live music venues in Melbourne through an intuitive and interactive map interface.







In [ ]:
# Creating the map base with Melbourne as the focal point with a 14 zoom frames the area of our dataset perfectly
map = folium.Map([-37.81368709240999, 144.95738102347036], zoom_start=14)

# Define the colours for each venue type
marker_colors = {
    'Nightclub': 'blue',
    'Accommodation': 'red',
    'Bar': 'green',
    'Museum': 'orange',
    'Venue': 'purple',
    'Hotel': 'yellow',
    'Performance space': 'cyan',
    'Casino': 'magenta',
    'Amusement/Theme Venue': 'lime',
    'Multi-use': 'brown',
    'Public University/College': 'pink',
    'Gallery': 'teal',
    'Theatre': 'gray',
    'Recording': 'olive',
    'Church': 'navy',
    'Community hall': 'maroon',
    'Library': 'white'
}

# Define the function to call and assign each colour according to the banner type
def style_function(feature):
    type_value = str(feature['properties']['banner_pole_type'])  # Convert to string
    color = marker_colors.get(type_value, 'black') # If no type is listed 'black' is the default
    return {'markerColor': color}

# Creation of the map
folium.GeoJson(
    geo_banner,
    name="Bookable Banner Poles",
    zoom_on_click=True, # When you click on a banner location the map will zoom and center on that location
    marker=folium.Marker(icon=folium.Icon(icon='star')), # This defines the shape of the icon and defines a 'star' within
    tooltip=folium.GeoJsonTooltip(fields=["asset_number", "banner_pole_type", "book_group"], aliases=["Asset Number", "Banner Type", "Booking Group"], labels=True, localize=True), # This describes what is displayed when the mouse hovers over a banner location
    popup=folium.GeoJsonPopup(fields=["asset_number", "banner_pole_type", "book_group"], aliases=["Asset Number", "Banner Type", "Booking Group"], labels=True, localize=True), # This describes what is displayed when a banner location is clicked on
    style_function=style_function # This calls the style function and facilitates the colours of each banner location
).add_to(map)

#_____________________________________________________________________

# Defining the function to call and assign each colour according to the banner type
def style_function(feature):
    type_value = str(feature['properties']['space_type'])  # Convert to string
    color = marker_colors.get(type_value, 'black') # If no type is listed 'black' is the default
    return {'markerColor': color}
# Allows the website link to be come interactive and able to be clicked on
geo_venue['website'] = '<a href="' + geo_venue.website + '">' + geo_venue.website + "</a>"

# Adding the second dataset to the map
folium.GeoJson(
    geo_venue,
    name="Live Music Venues",
    zoom_on_click=True, # When you click on a banner location the map will zoom and center on that location
    marker=folium.Marker(icon=folium.Icon(icon='star')), # This defines the shape of the icon and defines a 'star' within
    tooltip=folium.GeoJsonTooltip(fields=["venue_name", "space_type", "venue_address", "website"], aliases=["Venue Name", "Type", "Address", "Website"], labels=True, localize=True, parse_html=True), # This describes what is displayed when the mouse hovers over a banner location
    popup=folium.GeoJsonPopup(fields=["venue_name", "space_type", "venue_address", "website"], aliases=["Venue Name", "Type", "Address", "Website"], labels=True, localize=True, parse_html=True), # This describes what is displayed when a banner location is clicked on
    style_function=style_function, # This calls the style function and facilitates the colours of each banner location
    ).add_to(map)

# Display map
map

#Conclusion

In conclusion, the analysis of Melbourne's open datasets on bookable banner locations and live music venues has provided valuable insights into the city's cultural landscape. By visualising and combining these datasets, we've been able to identify intersections between public spaces designated for banners and areas bustling with live music activity. This integration not only offers a comprehensive view of the city's urban environment but also highlights potential opportunities for enhancing the vibrancy and visibility of Melbourne's music scene. The juxtaposition of banner locations with live music venues underscores the importance of strategic planning and collaboration in urban development initiatives, with the potential to foster community engagement and support for local businesses and cultural events. Overall, this analysis serves as a starting point for further exploration and initiatives aimed at enriching Melbourne's cultural fabric and promoting its dynamic arts and entertainment scene.

### What we Learnt from this Analysis

*   **Spatial Relationships**: By combining datasets on bookable banner locations and live music venues, we've identified spatial relationships between public spaces designated for banners and areas where live music activities thrive. This highlights potential synergies between urban infrastructure and cultural hubs.

*   **Community Engagement Opportunities**: The analysis presents opportunities for fostering community engagement and enhancing the visibility of Melbourne's music scene. Strategic placement of banners near live music venues can attract attention to cultural events and support local businesses.

*   **Urban Planning Implications**: Insights gained from this analysis can inform urban planning initiatives aimed at promoting cultural vitality and supporting the creative economy. It emphasises the importance of integrating cultural considerations into urban development strategies.

*   **Data-Driven Decision Making**: By leveraging open datasets and visualisation techniques, we've demonstrated the value of data-driven decision-making in understanding urban dynamics and fostering cultural development initiatives.








### References
**Dataset References**

[1] Live Music Venues: https://data.melbourne.vic.gov.au/explore/dataset/live-music-venues/information/

[2] Bookable Banner Poles: https://data.melbourne.vic.gov.au/explore/dataset/bookable-banner-poles/information/

<br></br>

**Technical References**

[1] Accessing geoJSON data https://stackoverflow.com/questions/48263802/finding-location-using-geojson-file-using-python

[2] Accessing geoJSON data https://medium.com/analytics-vidhya/measure-driving-distance-time-and-plot-routes-between-two-geographical-locations-using-python-39995dfea7e

[3] Visualising a geoJSON dataset https://python-visualization.github.io/folium/quickstart.html#GeoJSON/TopoJSON-Overlays